# Logistic Regression #


A retail bank wants to identitify to which of their existing stock trading customers can they offer a 10% discount on transaction commissions.

# Importing Libraries

In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np

# Reading Data

In [48]:
stockTradingData = pd.read_csv('Retail_Bank_Stock_trading_data.csv')

# Data Cleaning and Pre-Processing

In [49]:
stockTradingData.head()

,REF_NO,children,age_band,status,occupation,occupation_partner,home_status,family_income,self_employed,self_employed_partner,...,Investment Tax Saving Bond,Home Loan,Online Purchase Amount,Revenue Grid,gender,region,Investment in Commudity,Investment in Equity,Investment in Derivative,Portfolio Balance
0,1,Zero,51-55,Partner,Manual Worker,Secretarial/Admin,Own Home,"<17,500, >=15,000",No,No,...,19.99,0.00,0.00,1,Female,Wales,74.67,18.66,32.32,89.43
1,2,Zero,55-60,Single/Never Married,Retired,Retired,Own Home,"<27,500, >=25,000",No,No,...,0.00,0.00,0.00,2,Female,North West,20.19,0.00,4.33,22.78
2,3,Zero,26-30,Single/Never Married,Professional,Other,Own Home,"<30,000, >=27,500",Yes,No,...,0.00,3.49,0.00,2,Male,North,98.06,31.07,80.96,171.78
3,5,Zero,18-21,Single/Never Married,Professional,Manual Worker,Own Home,"<15,000, >=12,500",No,No,...,0.00,0.00,0.00,2,Female,West Midlands,4.10,14.15,17.57,-41.70
4,6,Zero,45-50,Partner,Business Manager,Unknown,Own Home,"<30,000, >=27,500",No,No,...,0.00,45.91,25.98,2,Female,Scotland,70.16,55.86,80.44,235.02


## Finding the datatype of each column

In [50]:
stockTradingData.dtypes

REF_NO                               int64
children                            object
age_band                            object
status                              object
occupation                          object
occupation_partner                  object
home_status                         object
family_income                       object
self_employed                       object
self_employed_partner               object
year_last_moved                      int64
TVarea                              object
post_code                           object
post_area                           object
Average Credit Card Transaction    float64
Balance Transfer                   float64
Term Deposit                       float64
Life Insurance                     float64
Medical Insurance                  float64
Average A/C Balance                float64
Personal Loan                      float64
Investment in Mutual Fund          float64
Investment Tax Saving Bond         float64
Home Loan  

In [51]:
""" Function to find the no of columns under each datatype """

def printDataType(df):
    dataTypes = ['object','int64','float64','bool','datetime64','timedelta[ns]','category']
    for dt in dataTypes:
        if np.sum(df.dtypes==dt) >0:
                print('Total number of columns with {} datatype '.format(dt), np.sum(df.dtypes == dt))

In [52]:
printDataType(stockTradingData)

Total number of columns with object datatype  14
Total number of columns with int64 datatype  3
Total number of columns with float64 datatype  15


## Converting all the object datatype to Int or Float

In [53]:
""" Function to find which columns are of 'object' datatype """

def printObjectDatatypeColumn(df):
    for name in df.columns:
        if df.loc[:,name].dtype == 'object':
            print(name)

In [54]:
printObjectDatatypeColumn(stockTradingData)

children
age_band
status
occupation
occupation_partner
home_status
family_income
self_employed
self_employed_partner
TVarea
post_code
post_area
gender
region


## Converting all the object datatype to Int or Float

### Convert the Output variable 'Revenue Grid' to either 0 or 1. 

In [55]:
stockTradingData['Revenue Grid'].head()

0    1
1    2
2    2
3    2
4    2
Name: Revenue Grid, dtype: int64

* In the 'Revenue Grid' column 1 means the customer is doing enough stock trading that, the bank would profit even after offerring a 10% discount on commission to that customer.
* 2 Means the customer is **not** doing enough stock trading that, the bank would **not** profit after offerring a 10% discount on commission to that customer.
* So, let's group the profitable customer's labelled as 1 into the a new label 1 and the not so profitable customer labelled as 1 into a new label 0. 
* Let's put the new labels '0' and '1' into a new column 'y'.
* This will help us classify the customers as 0 - Not eligible for discount, and 1 - eligible for 10% discount on commission.
* Finally, we'll drop the original column 'Revenue Grid' from the dataframe.

In [56]:
stockTradingData['y'] = np.where(stockTradingData['Revenue Grid']==2, 0, 1)

In [57]:
stockTradingData.drop(['Revenue Grid'], 1, inplace=True)

### Converting the column 'children'

In [58]:
stockTradingData['children'].value_counts()

Zero    6208
1       1848
2       1607
3        473
4+        19
Name: children, dtype: int64

In [59]:
""" converting 'zero' and '4+' to 0 and 4 respectively"""

stockTradingData.loc[stockTradingData['children'] == 'Zero','children'] =0
stockTradingData.loc[stockTradingData['children'] =='4+', 'children'] = 4

In [60]:
stockTradingData['children'].dtype

dtype('O')

In [61]:
"""After converting the 'zero' and '4+' to 0 and 4 we have now converted all
the strings in the column 'children' to integer. But still the datatype of 
that column remains as 'object'. 
So, we have to convert that column into an integer column """

stockTradingData['children'] = pd.to_numeric(stockTradingData['children'], errors='coerce')

In [62]:
printObjectDatatypeColumn(stockTradingData)

age_band
status
occupation
occupation_partner
home_status
family_income
self_employed
self_employed_partner
TVarea
post_code
post_area
gender
region


### Converting the column 'age_band' to several dummy variables

In [63]:
stockTradingData['age_band'].value_counts()

45-50      1359
36-40      1134
41-45      1112
31-35      1061
51-55      1052
55-60      1047
26-30       927
61-65       881
65-70       598
22-25       456
71+         410
18-21        63
Unknown      55
Name: age_band, dtype: int64

There are 13 different age bands. If we create dummy columns for 
each band then we will have to create 12 dummy columns. In order 
to reduce the number of dummy columns, let's group them together 
first with the output variable and then create dummies according
to the grouping

In [64]:
round((stockTradingData.groupby('age_band')['y']).mean(),2)

age_band
18-21      0.17
22-25      0.11
26-30      0.11
31-35      0.11
36-40      0.13
41-45      0.11
45-50      0.10
51-55      0.10
55-60      0.11
61-65      0.09
65-70      0.10
71+        0.10
Unknown    0.05
Name: y, dtype: float64

* Now, we have 6 groups[0.05,0.09,0.10,0.11,0.13,0.17] grouped according to their frequency of occurrence

* Let's keep the group 'Unknown' just like that. 

* Let's club the remaining age bands with the same mean value into the same group and create a new columns column for each group.


In [65]:
""" Grouping age_band with same frequencies into the same 
list and creating a corresponding string for each list. 

If the particular column we are working with contains values
in from one of the lists then those locations in the column
will be relaced with the corresponding strings we create for
each list """


firstBand = ['22-25','26-30','31-35','41-45','55-60']
firstBandGroup = 'ab_11'

secondBand = ['45-50','51-55','65-70','71+']
secondBandGroup = 'ab_10'

thirdBand = ['61-65']
thirdBandGroup = 'ab_9'

fourthBand = ['36-40']
fourthBandGroup = 'ab_13'

fifthBand = ['18-21']
fifthBandGroup= 'ab_17'

In [66]:
""" Creating two list of lists for ease of argument passing."""

bandListAgeBand = [firstBand,secondBand,thirdBand,fourthBand,fifthBand]
groupListAgeBand= [firstBandGroup, secondBandGroup, thirdBandGroup, fourthBandGroup, fifthBandGroup]

In [67]:
""" 
Function to replace values in a column with a string. 
The values in a column with the same frequency of occurrence 
were already grouped into lists. 

For each list we have also created a string. 

This function will find all positions in a column which has values 
contained in the list and will replace those values with the string
we already created corresponding to each list. 
"""

def groupDataByFrequency(df,columnName, bandList, groupList):
    for i in range(len(stockTradingData)):
        for j in range(len(bandList)):
            if stockTradingData.loc[i,columnName] in bandList[j]:
                stockTradingData.loc[i,columnName] = groupList[j]
            if stockTradingData.loc[i,columnName] in bandList[j]:
                stockTradingData.loc[i,columnName] = groupList[j]
            if stockTradingData.loc[i,columnName] in bandList[j]:
                stockTradingData.loc[i,columnName] = groupList[j]
            if stockTradingData.loc[i,columnName] in bandList[j]:
                stockTradingData.loc[i,columnName] = groupList[j]
            if stockTradingData.loc[i,columnName] in bandList[j]:
                stockTradingData.loc[i,columnName] = groupList[j]

In [68]:
groupDataByFrequency(stockTradingData, columnName= 'age_band', bandList=bandListAgeBand, groupList=groupListAgeBand)

In [69]:
stockTradingData['age_band'].value_counts()

ab_11      4603
ab_10      3419
ab_13      1134
ab_9        881
ab_17        63
Unknown      55
Name: age_band, dtype: int64

In [70]:
""" Now that we have reduced the groups in the data we can now create dummy variables"""

ageBandDummies = pd.get_dummies(stockTradingData['age_band'])
ageBandDummies.head()

,Unknown,ab_10,ab_11,ab_13,ab_17,ab_9
0,0,1,0,0,0,0
1,0,0,1,0,0,0
2,0,0,1,0,0,0
3,0,0,0,0,1,0
4,0,1,0,0,0,0


In [71]:
""" 
We can drop the column 'Unknown' from the ageBandDummies dataframe. 
It occurs only 55 times out of all the 10K+ values."""

ageBandDummies.drop(['Unknown'],axis=1,inplace=True)

In [72]:
ageBandDummies

,ab_10,ab_11,ab_13,ab_17,ab_9
0,1,0,0,0,0
1,0,1,0,0,0
2,0,1,0,0,0
3,0,0,0,1,0
4,1,0,0,0,0
...,...,...,...,...,...
10150,1,0,0,0,0
10151,1,0,0,0,0
10152,0,1,0,0,0
10153,1,0,0,0,0


In [73]:
""" Now, let's add this ageBandDummies dataframe back into the original dataframe"""

stockTradingData = stockTradingData.join(ageBandDummies)

In [74]:
""" Let's also drop the column 'age_band' from the stockTradingData dataframe"""

stockTradingData.drop(['age_band'], axis=1, inplace=True)

In [75]:
printObjectDatatypeColumn(stockTradingData)

status
occupation
occupation_partner
home_status
family_income
self_employed
self_employed_partner
TVarea
post_code
post_area
gender
region


### Converting the column 'occupation' to several dummy variables

In [76]:
stockTradingData['occupation'].value_counts()

Professional         2449
Retired              2206
Secretarial/Admin    1797
Housewife            1255
Business Manager      732
Unknown               567
Manual Worker         556
Other                 537
Student                56
Name: occupation, dtype: int64

In [77]:
round(stockTradingData.groupby('occupation')['y'].mean(),2)

occupation
Business Manager     0.12
Housewife            0.09
Manual Worker        0.11
Other                0.11
Professional         0.12
Retired              0.10
Secretarial/Admin    0.11
Student              0.11
Unknown              0.11
Name: y, dtype: float64

In [78]:
occFirstBand = ['Retired']
occFirstBandGroup = 'ocup_10'

occSecondBand = ['Manual Worker','Other','Secretarial/Admin','Student','Unknown']
occSecondBandGroup = 'ocup_11'

occThirdBand = ['Business Manager','Professional']
occThirdBandGroup = 'ocup_12'

In [79]:
occBand = [occFirstBand,occSecondBand,occThirdBand]
occGroup = [occFirstBandGroup,occSecondBandGroup,occThirdBandGroup]

In [80]:
groupDataByFrequency(stockTradingData,columnName='occupation', bandList=occBand, groupList=occGroup)

In [81]:
occDummies = pd.get_dummies(stockTradingData['occupation'])

In [82]:
stockTradingData=stockTradingData.join(occDummies)

In [83]:
stockTradingData.drop(['occupation','Housewife'], axis=1, inplace=True)

In [84]:
printObjectDatatypeColumn(stockTradingData)

status
occupation_partner
home_status
family_income
self_employed
self_employed_partner
TVarea
post_code
post_area
gender
region


### Converting the column 'occupation_partner' into several dummy variables

In [85]:
stockTradingData['occupation_partner'].value_counts()

Unknown              2394
Professional         2051
Retired              1936
Manual Worker        1508
Business Manager      737
Secretarial/Admin     662
Housewife             527
Other                 324
Student                16
Name: occupation_partner, dtype: int64

In [86]:
round(stockTradingData.groupby('occupation_partner')['y'].mean(),2)

occupation_partner
Business Manager     0.11
Housewife            0.11
Manual Worker        0.11
Other                0.10
Professional         0.11
Retired              0.10
Secretarial/Admin    0.12
Student              0.12
Unknown              0.10
Name: y, dtype: float64

In [87]:
partOccFirstBand = ['Other','Retired','Unknown']
partOccFirstBandGroup = 'partOcc_10'

partOccSecondBand = ['Business Manager','Housewife','Manual Worker','Professional']
partOccSecondBandGroup = 'partOcc_11'

partOccThirdBand = ['Secretarial/Admin','Student']
partOccThirdBandGroup = 'partOcc_12'

In [88]:
partOccBand = [partOccFirstBand,partOccSecondBand,partOccThirdBand]
partOccGroup = [partOccFirstBandGroup, partOccSecondBandGroup,partOccThirdBandGroup]

In [89]:
groupDataByFrequency(stockTradingData, columnName='occupation_partner', bandList=partOccBand, groupList=partOccGroup)

In [90]:
round(stockTradingData.groupby('occupation_partner')['y'].mean(),2)

occupation_partner
partOcc_10    0.10
partOcc_11    0.11
partOcc_12    0.12
Name: y, dtype: float64

In [92]:
stockTradingData.drop(['occupation_partner'], axis=1, inplace=True)

In [93]:
printObjectDatatypeColumn(stockTradingData)

status
home_status
family_income
self_employed
self_employed_partner
TVarea
post_code
post_area
gender
region


### Converting 'family_income'variable into mean family income

In [96]:
stockTradingData['family_income'].value_counts()

>=35,000             2517
<27,500, >=25,000    1227
<30,000, >=27,500     994
<25,000, >=22,500     833
<20,000, >=17,500     683
<12,500, >=10,000     677
<17,500, >=15,000     634
<15,000, >=12,500     629
<22,500, >=20,000     590
<10,000, >= 8,000     563
< 8,000, >= 4,000     402
< 4,000               278
Unknown               128
Name: family_income, dtype: int64

In [102]:
round(stockTradingData.groupby('family_income')['y'].mean(),2)

family_income
< 4,000              0.08
< 8,000, >= 4,000    0.08
<10,000, >= 8,000    0.11
<12,500, >=10,000    0.10
<15,000, >=12,500    0.11
<17,500, >=15,000    0.12
<20,000, >=17,500    0.11
<22,500, >=20,000    0.12
<25,000, >=22,500    0.10
<27,500, >=25,000    0.10
<30,000, >=27,500    0.12
>=35,000             0.11
Unknown              0.07
Name: y, dtype: float64

Since this is a currency related numeric variable, let's convert each band into the mean of the income band

Here we have three bands, '< 4,000', '>=35,000', and 'Unknown' instead of removing them we can just club them all together. 
An easy way to club it is to create a new variable called 'fi' with a default value of 4 At every location where the 'family_income' variable has a band, this new variable will have the average value of that band. Every other location will have a default value of 4. 

In [103]:
stockTradingData['fi'] = 4

In [109]:
stockTradingData.loc[stockTradingData['family_income'] == '< 8,000, >= 4,000', 'fi'] = (8+4)/2
stockTradingData.loc[stockTradingData['family_income'] == '<10,000, >= 8,000', 'fi'] = (10+8)/2
stockTradingData.loc[stockTradingData['family_income'] == '<12,500, >=10,000', 'fi'] = (12.5+10)/2
stockTradingData.loc[stockTradingData['family_income'] == '<15,000, >=12,500', 'fi'] = (15+12.5)/2
stockTradingData.loc[stockTradingData['family_income'] == '<17,500, >=15,000', 'fi'] = (17.5+15)/2
stockTradingData.loc[stockTradingData['family_income'] == '<20,000, >=17,500', 'fi'] = (20+17.5)/2
stockTradingData.loc[stockTradingData['family_income'] == '<22,500, >=20,000', 'fi'] = (22.5+20)/2
stockTradingData.loc[stockTradingData['family_income'] == '<25,000, >=22,500', 'fi'] = (25+22.5)/2
stockTradingData.loc[stockTradingData['family_income'] == '<27,500, >=25,000', 'fi'] = (27.5+25)/2
stockTradingData.loc[stockTradingData['family_income'] == '<30,000, >=27,500', 'fi'] = (30+27.5)/2

In [110]:
stockTradingData['fi']

0        16.25
1        26.25
2        28.75
3        13.75
4        28.75
         ...  
10150    28.75
10151    23.75
10152     4.00
10153     9.00
10154    26.25
Name: fi, Length: 10155, dtype: float64

In [112]:
stockTradingData['fi'].value_counts()

4.00     2923
26.25    1227
28.75     994
23.75     833
18.75     683
11.25     677
16.25     634
13.75     629
21.25     590
9.00      563
6.00      402
Name: fi, dtype: int64

In [114]:
stockTradingData.drop(['family_income'], axis=1, inplace=True)

In [115]:
printObjectDatatypeColumn(stockTradingData)

status
home_status
self_employed
self_employed_partner
TVarea
post_code
post_area
gender
region
